# | default_exp core

In [ ]:
# | hide
from bertopic import BERTopic
from bertopic.vectorizers import OnlineCountVectorizer
import dagshub
import dill as pickle
import dvc.api
from hdbscan import HDBSCAN
from itertools import tee, islice
import mlflow
import nbdev
from nbdev.showdoc import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import (
    CountVectorizer
    , TfidfTransformer
    , TfidfVectorizer
    ,
)
from src.custom_stanza_mlflow import StanzaWrapper
import src.dataframe_preprocessor as dfpp
import stanza
import tqdm
from umap import UMAP

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128'

# | export
def foo():
    pass

In [ ]:
# | hide
# this function allows us to get the experiment ID from an experiment name
def get_experiment_id(name):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
      exp_id = mlflow.create_experiment(name)
      return exp_id
    return exp.experiment_id

In [ ]:
# | hide
nbdev.nbdev_export()

# | Below this are blocks to use DagsHub with MLflow

In [ ]:
#@markdown Enter the username of your DAGsHub account:
DAGSHUB_USER_NAME = "AaronWChen" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
DAGSHUB_EMAIL = "awc33@cornell.edu" #@param {type:"string"}

#@markdown Enter the repo name 
DAGSHUB_REPO_NAME="MeaLeon"

#@markdown Enter the name of the branch you are working on 
BRANCH="STANZA-2/investigate_bertopic_compatibility"
dagshub.init(repo_name=DAGSHUB_REPO_NAME
             , repo_owner=DAGSHUB_USER_NAME)


Repository initialized!

In [ ]:
# instantiate stanza pipeline
stanza.download('en')
nlp = stanza.Pipeline('en', 
                      depparse_batch_size=50, 
                      depparse_min_length_to_batch_separately=50,
                      verbose=True,
                      use_gpu=False,
                    #   batch_size=100
                      )

2023-12-09 09:51:40 INFO: Downloading default packages for language: en (English) ...
2023-12-09 09:51:41 INFO: File exists: /home/awchen/stanza_resources/en/default.zip
2023-12-09 09:51:44 INFO: Finished downloading models and saved to /home/awchen/stanza_resources.
2023-12-09 09:51:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-12-09 09:51:45 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-12-09 09:51:45 INFO: Using device: cpu
2023-12-09 09:51:45 INFO: Loading: tokenize
2023-12-09 09:51:45 INFO: Loading: pos
2023-12-09 09:51:45 INFO: Loading: lemma
2023-12-09 09:51:45 INFO: Loading: constituency
2023-12-09 09:51:46 INFO: Loading: depparse
2023-12-09 09:51:46 INFO: Loading: sentiment
2023-12-09 09:51:46 INFO: Loading: ner
2023-12-09 09:51:46 INFO: Done loading processors!


In [ ]:
mlflow.set_tracking_uri(f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# starter idea for making an experiment name can be the git branch, but need more specificity
experiment_name = f"{DAGSHUB_EMAIL}/bertopic_stanza_small_set_v1"
mlflow_exp_id = get_experiment_id(experiment_name)

# run_id that has the logged info needed
mlflow_run_id = 'c6fbcf396af34ee3aade5503ee01c2bb'

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# load dataframes from artifacts
# mlflow.artifacts.download_artifacts(
#     run_id=mlflow_run_id
# )

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':3,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':5,
    'nr_topics':'auto',
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(cv_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=mlflow_exp_id):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print('Raw Dataframe:', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print('Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:50]
    print('\n')
    print('-' * 80)
    print('Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params,
    )

    def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength):
            lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

            preproc = stanza_pipeline(lowered)
            
            lemmad = " ".join(map(str,
                                [word.text
                                for sent in preproc.sentences 
                                for word in sent.words if (
                                    word is not None
                                )]
                            )
                        )
            
            # analyze each line of the input string seperately
            for ln in lemmad.split(' brk '):
                
                # tokenize the input string (customize the regex as desired)
                at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
                terms = re.split(at_least_two_english_characters_whole_words, ln)

                # loop ngram creation for every number between min and max ngram length
                for ngramLength in range(minNgramLength, maxNgramLength+1):

                    # find and return all ngrams
                    # for ngram in zip(*[terms[i:] for i in range(3)]): 
                        # <-- solution without a generator (works the same but has higher memory usage)
                    for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                        
                        ngram = ' '.join(map(str, ngram))
                        # yield ngram
                        return str(ngram)

    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_steps = to_nlp_df["prepSteps"].apply(custom_analyzer, **analyzer_kwargs)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print('Recipe steps:', end='\n')
    print(recipe_steps)

    # train on the recipes' steps
    topics, probs = topic_model.fit_transform(recipe_steps)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    # steps_vectorizer_model = CountVectorizer(**cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    steps_vectorizer_model = OnlineCountVectorizer(**cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_steps
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print('BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_small_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_small_set_df.json',
                        artifact_path='bertopic_models')




--------------
Raw Dataframe:
                         id  \
0  54a2b6b019925f464b373351   
1  54a408a019925f464b3733bc   
2  54a408a26529d92b2c003631   
3  54a408a66529d92b2c003638   
4  54a408a719925f464b3733cc   

                                                 dek  \
0  How does fried chicken achieve No. 1 status? B...   
1                                Spinaci all'Ebraica   
2  This majestic, moist, and richly spiced honey ...   
3  The idea for this sandwich came to me when my ...   
4  In 1930, Simon Agranat, the chief justice of t...   

                                     hed                   pubDate  \
0            Pickle-Brined Fried Chicken  2014-08-19T04:00:00.000Z   
1                   Spinach Jewish Style  2008-09-09T04:00:00.000Z   
2                  New Year’s Honey Cake  2008-09-10T04:00:00.000Z   
3  The B.L.A.Bagel with Lox and Avocado  2008-09-08T04:00:00.000Z   
4        Shakshuka a la Doktor Shakshuka  2008-09-09T04:00:00.000Z   

                       

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2023-12-09 10:34:55,903 - BERTopic - Transformed documents to Embeddings
2023-12-09 10:34:57,257 - BERTopic - Reduced dimensionality
2023-12-09 10:34:57,266 - BERTopic - Clustered reduced embeddings
2023-12-09 10:34:57,288 - BERTopic - Reduced number of topics from 3 to 3


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 106167298048 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# load dataframes from artifacts
# mlflow.artifacts.download_artifacts(
#     run_id=mlflow_run_id
# )

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':5,
    'nr_topics':'auto',
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(cv_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_stanza_ingreds_full_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print('Raw Dataframe:', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print('Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    # to_nlp_df = pre_proc_df[0:50]
    # print('\n')
    # print('-' * 80)
    # print('Subset Dataframe:', end='\n')
    # print(to_nlp_df.head())
    # print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params,
    )

    def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength):
            lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

            preproc = stanza_pipeline(lowered)
            
            lemmad = " ".join(map(str,
                                [word.text
                                for sent in preproc.sentences 
                                for word in sent.words if (
                                    word is not None
                                )]
                            )
                        )
            
            # analyze each line of the input string seperately
            for ln in lemmad.split(' brk '):
                
                # tokenize the input string (customize the regex as desired)
                at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
                terms = re.split(at_least_two_english_characters_whole_words, ln)

                # loop ngram creation for every number between min and max ngram length
                for ngramLength in range(minNgramLength, maxNgramLength+1):

                    # find and return all ngrams
                    # for ngram in zip(*[terms[i:] for i in range(3)]): 
                        # <-- solution without a generator (works the same but has higher memory usage)
                    for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                        
                        ngram = ' '.join(map(str, ngram))
                        # yield ngram
                        return str(ngram)

    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = pre_proc_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print('Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' steps
    topics, probs = topic_model.fit_transform(recipe_ingreds)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    # steps_vectorizer_model = CountVectorizer(**cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    steps_vectorizer_model = OnlineCountVectorizer(**cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print('BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')




--------------
Raw Dataframe:
                         id  \
0  54a2b6b019925f464b373351   
1  54a408a019925f464b3733bc   
2  54a408a26529d92b2c003631   
3  54a408a66529d92b2c003638   
4  54a408a719925f464b3733cc   

                                                 dek  \
0  How does fried chicken achieve No. 1 status? B...   
1                                Spinaci all'Ebraica   
2  This majestic, moist, and richly spiced honey ...   
3  The idea for this sandwich came to me when my ...   
4  In 1930, Simon Agranat, the chief justice of t...   

                                     hed                   pubDate  \
0            Pickle-Brined Fried Chicken  2014-08-19T04:00:00.000Z   
1                   Spinach Jewish Style  2008-09-09T04:00:00.000Z   
2                  New Year’s Honey Cake  2008-09-10T04:00:00.000Z   
3  The B.L.A.Bagel with Lox and Avocado  2008-09-08T04:00:00.000Z   
4        Shakshuka a la Doktor Shakshuka  2008-09-09T04:00:00.000Z   

                       

Batches:   0%|          | 0/1083 [00:00<?, ?it/s]

2023-12-10 07:16:59,080 - BERTopic - Transformed documents to Embeddings
2023-12-10 07:17:36,106 - BERTopic - Reduced dimensionality
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` befo

: 